In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import os
import fnmatch
import pandas as pd
import warnings
from PIL import Image
import os
warnings.filterwarnings('ignore')
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import os
import cv2
import datetime
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import regularizers

from tensorflow.keras.optimizers import legacy as legacy_optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import regularizers
import pickle

In [ ]:
class_names = np.load("class_names.npy", allow_pickle=True).item()

train_df = pd.read_csv("train_images.csv")

train_datagen = ImageDataGenerator(rescale=1./255)

current_dir = os.getcwd()

train_images = os.path.join(current_dir, 'train_images')

X = []
y = []

for index, row in train_df.iterrows():
    img_filename = row['image_path'][1:]  
    img_path = os.path.join(train_images, img_filename)
    label = row['label'] - 1

    img = cv2.imread(img_path)
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (299, 299))
    img = img / 255.0

    X.append(img)
    y.append(label)

X_train = np.array(X)
y_train = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)

In [ ]:
### HYPERPARAMETER TUNING

def create_model(optimizer=legacy_optimizers.Adam(lr=0.001), neurons=256, reg_strength=0.01, learning_rate=0.001):
 
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(neurons, activation='relu', kernel_regularizer=regularizers.l2(reg_strength)))
    model.add(BatchNormalization())
    model.add(Dense(len(class_names), activation='softmax'))

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


param_grid = {
    'neurons': [512], # 128, 256
    'reg_strength': [0.01, 0.001],
    'learning_rate': [0.001, 0.01]
}

best_score = 0
best_params = {}

for neurons in param_grid['neurons']:
    for reg_strength in param_grid['reg_strength']:
        for learning_rate in param_grid['learning_rate']:
           
            model = create_model(neurons=neurons, reg_strength=reg_strength, learning_rate=learning_rate) 

            # Train the model
            history = model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=1, validation_data=(X_val, y_val))
            val_accuracy = history.history['val_accuracy'][0]

            if val_accuracy > best_score:
                best_score = val_accuracy
                best_params = {'neurons': neurons, 'reg_strength': reg_strength, 'learning_rate': learning_rate}
                print("Validation accuracy: {:.4f} for {}".format(val_accuracy, best_params))

print("Best validation accuracy: {:.4f} using {}".format(best_score, best_params))


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization()) 
model.add(Dense(len(class_names), activation='softmax'))  

model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(X_train, y_train, epochs=2, validation_data=(X_val, y_val), callbacks=[early_stopping]) 

In [ ]:
with open('model20.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open('model20.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:

test_df = pd.read_csv("test_images_path.csv")
current_dir = os.getcwd()
test_datagen = ImageDataGenerator(rescale=1./255)

test_images = os.path.join(current_dir, 'test_images')  

X_test = []


for index, row in test_df.iterrows():
    img_filename = row['image_path'][1:]  
    img_path = os.path.join(test_images, img_filename)

    img = cv2.imread(img_path)
    

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img = cv2.resize(img, (299, 299))
    img = img / 255.0

    X_test.append(img)

X_test = np.array(X_test)

predictions = model.predict(X_test)

predicted_labels = np.argmax(predictions, axis=1)

test_df['label'] = predicted_labels

selected_columns = ['id', 'label']
test_df_selected = test_df[selected_columns]

current_datetime = datetime.datetime.now()
timestamp = current_datetime.strftime("%Y%m%d_%H%M")
file_name = f"submission_{timestamp}.csv"

submissions_folder = os.path.join(current_dir, 'submissions')  

file_path = os.path.join(submissions_folder, file_name)

test_df_selected.to_csv(file_path, index=False)